# 한국어 텍스트 감정 분석 - 데이터 전처리

이 노트북은 한국어 영화 리뷰 감정 분석을 위한 데이터 전처리 및 탐색적 데이터 분석(EDA)을 수행합니다.

## 주요 기능
- 데이터 품질 검사 및 정제
- 텍스트 전처리 파이프라인
- 훈련/검증 데이터 분할
- 전처리된 데이터 CSV 저장


### IMPORT

In [42]:
# 한국어 텍스트 감정 분석을 위한 필수 라이브러리들
from collections import Counter  # 카운터 자료구조
import os  # 운영체제 인터페이스
import platform  # 플랫폼 정보
import re  # 정규 표현식
import sys  # 시스템 정보
import warnings  # 경고 메시지 제어
import math


import matplotlib.pyplot as plt  # 데이터 시각화
plt.rc("font", family="NanumBarunGothic")  # 한글 폰트 설정(없으면 설치 필요)

import numpy as np  # 수치 연산
import pandas as pd  # 데이터 처리 및 분석
import seaborn as sns  # 고급 시각화
import torch  # 딥러닝 프레임워크
import koreanize_matplotlib
import wandb
# 머신러닝 관련 라이브러리
from sklearn.metrics import accuracy_score, f1_score  # 평가 지표
from sklearn.model_selection import train_test_split # 데이터 분할

# 트랜스포머 및 BERT 관련 라이브러리
from transformers import (
    AutoModelForSequenceClassification,  # 시퀀스 분류 모델
    AutoTokenizer,  # 토크나이저
    DataCollatorWithPadding,  # 패딩 데이터 콜레이터
    set_seed,
    Trainer,  # 트레이너
    TrainingArguments,  # 훈련 설정
)

# PyTorch 데이터 처리
from torch.utils.data import Dataset  # 데이터셋 및 데이터로더

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

# 라이브러리 버전 정보 출력 (재현성을 위함)
print("=== 라이브러리 버전 정보 ===")
print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"torch: {torch.__version__}")
print(f"transformers: {__import__('transformers').__version__}")
print(f"sklearn: {__import__('sklearn').__version__}")
print(f"matplotlib: {__import__('matplotlib').__version__}")
print(f"seaborn: {sns.__version__}")

# GPU 사용 가능 여부 확인
print("\n=== PyTorch GPU 지원 정보 ===")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.current_device()}")
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
else:
    print("CPU에서 실행 중")


=== 라이브러리 버전 정보 ===
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Platform: Linux-5.4.0-99-generic-x86_64-with-glibc2.31
pandas: 2.2.3
numpy: 1.26.4
torch: 2.6.0+cu124
transformers: 4.55.0
sklearn: 1.6.1
matplotlib: 3.10.5
seaborn: 0.13.2

=== PyTorch GPU 지원 정보 ===
CUDA 사용 가능: True
CUDA 버전: 12.4
GPU 개수: 1
현재 GPU: 0
GPU 이름: Tesla V100-SXM2-32GB


In [43]:
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()


True

## Random Seed Configuration


In [44]:
# 랜덤 시드 설정
RANDOM_STATE = 42


set_seed(RANDOM_STATE)

print(f"랜덤 시드 {RANDOM_STATE}로 설정 완료")


랜덤 시드 42로 설정 완료


# Data Load


In [45]:
# 데이터 로드
df = pd.read_csv("data/train.csv")

# 처음 몇 행 표시
print("\n처음 5행:")
df.head()



처음 5행:


,ID,review,label,type
0,0,이 영화는 정말 여성의 강인함과 힘을 제대로 보여주는 작품이었어요! 주인공이 자기 ...,2,augment
1,1,어느 부잣집 도련님의 철없는 행각,1,original
2,3,왜이렇게 재미가없냐 원도 별로였지만 원보다 더 재미없네,0,original
3,4,크리스마스 시즌엔 무조건 홈 알론이죠! 맥컬리 컬킨이 연기한 케빈의 재치있고 천방지...,2,augment
4,5,참나ㅋㅋ이게 무슨 드라마 최초 뮤지컬드라마야 이게무슨ㅋㅋ걍 다른 드라마랑 똑같구만 ...,0,original


# Data/Feature Engineering


In [46]:
# 데이터 전처리를 위한 복사본 생성
df_processed = df[["ID", "label", "review"]].copy()

print(f"원본 데이터셋 크기: {len(df_processed):,}개")

df_processed.head()


원본 데이터셋 크기: 279,650개


,ID,label,review
0,0,2,이 영화는 정말 여성의 강인함과 힘을 제대로 보여주는 작품이었어요! 주인공이 자기 ...
1,1,1,어느 부잣집 도련님의 철없는 행각
2,3,0,왜이렇게 재미가없냐 원도 별로였지만 원보다 더 재미없네
3,4,2,크리스마스 시즌엔 무조건 홈 알론이죠! 맥컬리 컬킨이 연기한 케빈의 재치있고 천방지...
4,5,0,참나ㅋㅋ이게 무슨 드라마 최초 뮤지컬드라마야 이게무슨ㅋㅋ걍 다른 드라마랑 똑같구만 ...


In [34]:
def clean_data_quality(df, text_col='review', label_col='label', id_col='ID'):
    """
    데이터 품질 검사 및 문제가 있는 데이터 제거
    
    Args:
        df: 원본 데이터프레임
        text_col: 텍스트 컬럼명
        label_col: 라벨 컬럼명  
        id_col: ID 컬럼명
    
    Returns:
        clean_df: 품질 문제가 제거된 데이터프레임
        removed_info: 제거된 데이터 정보
    """
    print(f"원본 데이터 수: {len(df):,}개")
    
    # 1. 품질 검사
    null_mask = df[text_col].isnull()
    empty_mask = df[text_col].str.strip().eq("")
    whitespace_mask = df[text_col].str.isspace()
    duplicate_mask = df[text_col].duplicated()
    digit_only_mask = df[text_col].str.match(r"^\d+$", na=False)
    
    print(f"Null 값: {null_mask.sum()}개")
    print(f"빈 텍스트: {empty_mask.sum()}개") 
    print(f"공백만 있는 텍스트: {whitespace_mask.sum()}개")
    print(f"중복 리뷰: {duplicate_mask.sum()}개")
    print(f"숫자만 있는 리뷰: {digit_only_mask.sum()}개")
    
    # 2. 품질 문제가 있는 데이터 마스크 생성
    quality_issues_mask = (
        null_mask | 
        empty_mask | 
        whitespace_mask | 
        duplicate_mask | 
        digit_only_mask 
    )
    
    # 3. 품질 문제가 있는 데이터 제거
    clean_df = df[~quality_issues_mask].copy().reset_index(drop=True)
    removed_count = quality_issues_mask.sum()
    
    print(f"\n=== 데이터 품질 검사 결과 ===")
    print(f"제거된 데이터: {removed_count:,}개")
    print(f"남은 데이터: {len(clean_df):,}개")
    print(f"데이터 품질 비율: {len(clean_df) / len(df) * 100:.2f}%")
    
    # 4. 제거된 데이터의 상세 정보 출력
    if removed_count > 0:
        print(f"\n=== 제거된 데이터 상세 정보 ===")
        removed_data = df[quality_issues_mask]
        
        if null_mask.sum() > 0:
            print(f"Null 값 샘플: {removed_data[null_mask][text_col].head(3).tolist()}")
        if empty_mask.sum() > 0:
            print(f"빈 텍스트 샘플: {removed_data[empty_mask][text_col].head(3).tolist()}")
        if whitespace_mask.sum() > 0:
            print(f"공백만 있는 텍스트 샘플: {removed_data[whitespace_mask][text_col].head(3).tolist()}")
        if duplicate_mask.sum() > 0:
            print(f"중복 리뷰 샘플: {removed_data[duplicate_mask][text_col].head(3).tolist()}")
        if digit_only_mask.sum() > 0:
            print(f"숫자만 있는 리뷰 샘플: {removed_data[digit_only_mask][text_col].head(3).tolist()}")
    
    # 5. 제거된 데이터 정보 저장
    removed_info = {
        'total_removed': removed_count,
        'null_count': null_mask.sum(),
        'empty_count': empty_mask.sum(),
        'whitespace_count': whitespace_mask.sum(),
        'duplicate_count': duplicate_mask.sum(),
        'digit_only_count': digit_only_mask.sum()
    }
    
    return clean_df, removed_info

print("✅ 데이터 품질 검사 및 제거 함수 정의 완료")


✅ 데이터 품질 검사 및 제거 함수 정의 완료


In [35]:
# 텍스트 전처리 파이프라인 클래스 구성
class TextPreprocessingPipeline:
    """
    텍스트 전처리 파이프라인 클래스
    - 기본 전처리와 학습 데이터 기반 고급 전처리를 통합 관리
    - 재사용 가능하고 확장 가능한 구조
    """

    def __init__(self):
        self.is_fitted = False
        self.vocab_info = {}
        self.label_patterns = {}

    def basic_preprocess(self, texts):
        """기본 전처리 (clean_text + normalize 기능)"""
        processed_texts = []
        for text in texts:
            # 기본 텍스트 정리
            cleaned = self._clean_text(text)
            processed_texts.append(cleaned)
        return processed_texts

    def _clean_text(self, text):
        """기존 clean_text 함수 내용"""
        if pd.isna(text):
            return ""

        text = str(text).strip()
        text = text.lower() # 소문자 변환
        text = self._remove_urls_emails_mentions(text) # URL, 이메일, 멘션 제거
        text = self._normalize_punctuation(text)  # 구두점 정규화
        #text = self._remove_incomplete_korean(text)
        text = self._normalize_emotion_expressions(text) # 감정 표현 정규화 (ㅋㅋㅋ , ㅎㅎㅎ)
        text = self._reduce_excessive_repetition(text) # 과도한 문자 반복 축소 (아아아아아아앙 -> 아아아아)
        text = self._clean_special_characters(text) # 특수문자 제거 (이모티콘, 특수기호)
        text = self._normalize_whitespace(text) # 공백 정규화 (여러 개의 공백 -> 하나의 공백)

        return text.strip()

    def fit(self, texts, labels=None):
        """학습 데이터로부터 전처리 정보 학습 (품질 검사 기준 학습)"""

        self.is_fitted = True
        print("✓ 전처리 파이프라인 학습 완료")


    def transform(self, texts):
        """전처리 적용 (품질 문제 데이터 제거 + 텍스트 전처리)"""
        if not self.is_fitted:
            print(
                "Warning: 파이프라인이 학습되지 않았습니다. 기본 전처리만 적용합니다."
            )
            return self.basic_preprocess(texts)
        
        # 텍스트 전처리 적용
        return self.basic_preprocess(texts)

    def fit_transform(self, texts, labels=None):
        """학습과 변환을 동시에 수행"""
        # 1. 학습 단계 (품질 검사 기준 학습)
        self.fit(texts, labels)
        
        # 2. 변환 단계 (품질 문제 데이터 제거 + 텍스트 전처리)
        processed_texts = self.transform(texts)
        
        # 3. 라벨도 동일하게 필터링
        return processed_texts


    @staticmethod
    def _remove_incomplete_korean(text):
        """불완전한 한글 제거 (자음/모음만 있는 경우)"""
        return re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", "", text)

    @staticmethod
    def _normalize_emotion_expressions(text):
        """감정 표현 정규화"""
        def replace_emotion(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환
            new_count = int(math.log2(count)) + 1 if count > 0 else 1
            return char * new_count
        
        # 웃음과 슬픔 표현 정규화 (2번 이상 반복)
        text = re.sub(r"([ㅋㅎ])\1+", replace_emotion, text)
        text = re.sub(r"([ㅠㅜㅡ])\1+", replace_emotion, text)
        return text

    @staticmethod
    def _reduce_excessive_repetition(text):
        """과도한 문자 반복 축소 (4번 이상 → 3번으로)"""
        
        def replace_repetition(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환하고 최소 1개 보장
            new_count = max(1, int(math.log2(count)) + 1) if count > 0 else 1
            return char * new_count
        
        return re.sub(r"(.)\1{3,}", replace_repetition, text)

    @staticmethod
    def _clean_special_characters(text):
        """특수문자 제거 (이모티콘 보존)"""

        # 1. 허용할 이모티콘 범위 정의
        # emoji_ranges = r"\U0001F600-\U0001F64F"  # Emoticons
        # emoji_ranges += r"\U0001F300-\U0001F5FF"  # Misc Symbols/Pictographs
        # emoji_ranges += r"\U0001F680-\U0001F6FF"  # Transport/Map
        # emoji_ranges += r"\U00002600-\U000026FF"  # Misc Symbols (★ 포함)
        # emoji_ranges += r"\U00002700-\U000027BF"  # Dingbats

        # 2. 허용할 기타 특수기호 정의
        other_symbols = r"@★#$" # 예시로 @ 추가

        # 3. 허용할 문자들을 조합하여 정규식 생성
        #allowed_chars = rf"\w\s가-힣.,!?ㅋㅎㅠㅜㅡ~\-{emoji_ranges}{other_symbols}"
        allowed_chars = rf"\w\s가-힣.,!?ㅋㅎㅠㅜㅡ~\-"
        
        return re.sub(rf"[^{allowed_chars}]", " ", text)
 
 
    @staticmethod
    def _normalize_whitespace(text):
        """공백 정규화"""
        return re.sub(r"\s+", " ", text)

    @staticmethod
    def _normalize_punctuation(text):
        """구두점 정규화 (log 공식 적용)"""
        def replace_punctuation(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환
            new_count = int(math.log2(count)) + 1 if count > 0 else 1
            return char * new_count
        
        # 각 구두점별로 log 공식 적용
        text = re.sub(r"([.])\1+", replace_punctuation, text)
        text = re.sub(r"([!])\1+", replace_punctuation, text)
        text = re.sub(r"([?])\1+", replace_punctuation, text)
        text = re.sub(r"([,])\1+", replace_punctuation, text)
        
        # 구두점 앞뒤 공백 정리
        text = re.sub(r"\s+([.,!?])", r"\1", text)
        text = re.sub(r"([.,!?])\s+", r"\1 ", text)
        
        return text

    @staticmethod
    def _remove_urls_emails_mentions(text):
        """URL, 이메일, 멘션 제거"""
        # URL 패턴 제거
        text = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
            "",
            text,
        )
        # 이메일 패턴 제거
        text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b", "", text)
        # 멘션 패턴 제거
        text = re.sub(r"\B@\w+", "", text)
        
        return text


In [36]:
preprocessor = TextPreprocessingPipeline()


## 데이터 분할 전략

- 훈련/검증 데이터 분할
- 클래스 분포를 유지하는 계층적 분할


In [37]:
# 데이터 품질 검사 및 제거
df_clean, removed_info = clean_data_quality(df_processed)

print(f"\n원본 데이터: {len(df_processed):,}개")
print(f"정제된 데이터: {len(df_clean):,}개")
print(f"제거된 데이터: {removed_info['total_removed']:,}개")


원본 데이터 수: 279,650개
Null 값: 6개
빈 텍스트: 0개
공백만 있는 텍스트: 0개
중복 리뷰: 3350개
숫자만 있는 리뷰: 64개

=== 데이터 품질 검사 결과 ===
제거된 데이터: 3,379개
남은 데이터: 276,271개
데이터 품질 비율: 98.79%

=== 제거된 데이터 상세 정보 ===
Null 값 샘플: [nan, nan, nan]
중복 리뷰 샘플: ['재미없다', '굳', 'tv 전기세가 아깝다!!!']
숫자만 있는 리뷰 샘플: ['1234567890', '2', '1']

원본 데이터: 279,650개
정제된 데이터: 276,271개
제거된 데이터: 3,379개


In [38]:
# 데이터 분할 - 텍스트 전처리 파이프라인 적용
X = df_clean["review"]  # 원본 텍스트 데이터 사용 (파이프라인에서 전처리 수행)
y = df_clean["label"]
ids = df_clean["ID"]

# 훈련/검증 데이터 분할 (train 80%, val 20%) - 계층 분할로 클래스 분포 유지
X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(
    X, y, ids, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"전체 데이터: {len(X):,}개")
print(f"훈련 데이터: {len(X_train):,}개")
print(f"검증 데이터: {len(X_val):,}개")

# 텍스트 전처리 파이프라인 적용
print("훈련 데이터에 대한 전처리 파이프라인 학습 및 적용...")
X_train_processed = preprocessor.fit_transform(X_train.tolist(), y_train.tolist())

print("검증 데이터에 전처리 파이프라인 적용...")
X_val_processed = preprocessor.transform(X_val.tolist())

# 원본 데이터프레임 구조로 분할된 데이터 생성 - 모델 학습용 형태
train_data = pd.DataFrame(
    {"ID": ids_train, "review": X_train_processed, "label": y_train}
).reset_index(drop=True)

val_data = pd.DataFrame(
    {"ID": ids_val, "review": X_val_processed, "label": y_val}
).reset_index(drop=True)

print(f"\nTrain: {len(train_data)}, Val: {len(val_data)}")

# 계층 분할이 올바르게 수행되었는지 검증 - 클래스 분포 확인
print("\n클래스 분포 검증:")
print("원본 데이터:")
original_distribution = y.value_counts(normalize=True).sort_index()
original_counts = y.value_counts().sort_index()
for idx, val in original_distribution.items():
    count = original_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

print("\n훈련 데이터:")
train_distribution = train_data["label"].value_counts(normalize=True).sort_index()
train_counts = train_data["label"].value_counts().sort_index()
for idx, val in train_distribution.items():
    count = train_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

print("\n검증 데이터:")
val_distribution = val_data["label"].value_counts(normalize=True).sort_index()
val_counts = val_data["label"].value_counts().sort_index()
for idx, val in val_distribution.items():
    count = val_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

# 전처리 결과 확인
print("\n전처리 결과 샘플:")
for i in range(3):
    print(f"원본: {X_train.iloc[i]}")
    print(f"전처리: {X_train_processed[i]}")
    print()


전체 데이터: 276,271개
훈련 데이터: 221,016개
검증 데이터: 55,255개
훈련 데이터에 대한 전처리 파이프라인 학습 및 적용...
✓ 전처리 파이프라인 학습 완료
검증 데이터에 전처리 파이프라인 적용...

Train: 221016, Val: 55255

클래스 분포 검증:
원본 데이터:
  클래스 0: 112866개 (40.9%)
  클래스 1: 26917개 (9.7%)
  클래스 2: 97894개 (35.4%)
  클래스 3: 38594개 (14.0%)

훈련 데이터:
  클래스 0: 90292개 (40.9%)
  클래스 1: 21534개 (9.7%)
  클래스 2: 78315개 (35.4%)
  클래스 3: 30875개 (14.0%)

검증 데이터:
  클래스 0: 22574개 (40.9%)
  클래스 1: 5383개 (9.7%)
  클래스 2: 19579개 (35.4%)
  클래스 3: 7719개 (14.0%)

전처리 결과 샘플:
원본: 아 진짜 요즘 영화관에서 본 영화 중에 제일 재미있었음! 시사회로 먼저 보게 됐는데, 보는 내내 소름 돋고 심장 뛰는 경험을 했다는 게 믿기지 않아. 평점이 높다고 해서 좀 기대하긴 했는데, 그 기대 이상의 퀄리티를 보여준 것 같아서 너무 좋았어. 특히 주인공의 연기는 정말 소름 끼치도록 진짜 같았고, 스토리의 흐름도 너무 자연스러워서 보는 내내 몰입감 최고였어. 진짜로 이 영화를 보는 내내 내가 그 현장에 있는 것 같은 느낌이 들 정도로 연출이 미쳤다고 생각함. 평이 대단하다고 해서 좀 걱정했는데, 실제로 보고 나니까 왜 그렇게 평이 좋은지 완전 이해됨. 아직도 그 감동과 여운이 남아 있어서, 다시 한번 보고 싶은 생각이 간절해. 이 영화를 보면서 느낀 게 많은데, 그 중에서도 가장 크게 느낀 건 이런 좋은 영화를 만나게 해준 감독님과 배우분들께 너무 감사하다는 거야. 진짜로 이 영화는 꼭 한번 봐야 할 영화라고 생각함!
전처리: 아 진짜 요즘 영화관에서 본 영화 중에 제일 재미있었음! 시사회로 먼저

# 데이터 저장

전처리된 데이터를 CSV 파일로 저장하여 모델 학습 노트북에서 사용할 수 있도록 합니다.


In [39]:
# 전처리된 데이터를 CSV 파일로 저장
import os

# data 디렉토리가 없으면 생성
os.makedirs("data", exist_ok=True)

# 훈련 데이터 저장
train_data.to_csv("data/train_processed.csv", index=False)
print(f"✅ 훈련 데이터 저장 완료: data/train_processed.csv ({len(train_data):,}개)")

# 검증 데이터 저장
val_data.to_csv("data/val_processed.csv", index=False)
print(f"✅ 검증 데이터 저장 완료: data/val_processed.csv ({len(val_data):,}개)")

# 전처리 파이프라인 정보 저장 (선택사항)
preprocessing_info = {
    'total_original_samples': len(df),
    'total_clean_samples': len(df_clean),
    'removed_samples': removed_info['total_removed'],
    'train_samples': len(train_data),
    'val_samples': len(val_data),
    'preprocessing_pipeline_fitted': preprocessor.is_fitted
}

print(f"\n=== 전처리 완료 요약 ===")
print(f"원본 데이터: {preprocessing_info['total_original_samples']:,}개")
print(f"품질 검사 후: {preprocessing_info['total_clean_samples']:,}개")
print(f"제거된 데이터: {preprocessing_info['removed_samples']:,}개")
print(f"훈련 데이터: {preprocessing_info['train_samples']:,}개")
print(f"검증 데이터: {preprocessing_info['val_samples']:,}개")
print(f"전처리 파이프라인 학습 완료: {preprocessing_info['preprocessing_pipeline_fitted']}")

print(f"\n✅ 전처리 완료! 다음 단계: train_pytorch.ipynb에서 모델 학습을 진행하세요.")


✅ 훈련 데이터 저장 완료: data/train_processed.csv (221,016개)
✅ 검증 데이터 저장 완료: data/val_processed.csv (55,255개)

=== 전처리 완료 요약 ===
원본 데이터: 279,650개
품질 검사 후: 276,271개
제거된 데이터: 3,379개
훈련 데이터: 221,016개
검증 데이터: 55,255개
전처리 파이프라인 학습 완료: True

✅ 전처리 완료! 다음 단계: train_pytorch.ipynb에서 모델 학습을 진행하세요.


In [47]:
test_data = pd.read_csv("data/test.csv")

test_data.head()

test_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59928 entries, 0 to 59927
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      59928 non-null  int64 
 1   review  59927 non-null  object
dtypes: int64(1), object(1)
memory usage: 936.5+ KB


In [50]:
processed_test_data = preprocessor.transform(test_data["review"].tolist())

processed_test_data

test_data["review"] = processed_test_data

test_data.head()

test_data.to_csv("data/test_processed.csv", index=False)



